In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

read data 

In [5]:
prices=pd.read_csv("data/2020-01-01-prices.csv")
stations=pd.read_csv("data/stations.csv")

In [6]:
prices.head()

,date,station_uuid,diesel,e5,e10,dieselchange,e5change,e10change
0,2020-01-01 00:00:04+01,81d72fa3-5270-45af-816f-ba0abb7776f4,1.319,1.469,1.449,1,0,0
1,2020-01-01 00:01:04+01,ec70685a-1110-4632-8735-8d2cebd07da6,1.269,1.389,1.369,1,1,1
2,2020-01-01 00:01:04+01,4de95bf1-c91c-4bf9-a0f3-9c1069dfc277,1.289,1.419,1.399,1,1,1
3,2020-01-01 00:01:04+01,9bb515a4-c551-41b8-a8a1-e120f29474fd,1.229,1.359,0.000,1,1,0
4,2020-01-01 00:01:04+01,3e385667-56a9-46b4-96de-2adcfd7b5655,1.319,1.439,1.419,1,1,1


In [7]:
stations.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude
0,00060723-0001-4444-8888-acdc00000001,BAGeno Raiffeisen eG,NaN,Künzelsauer Strasse,7,74653,Ingelfingen,49.296822,9.661385
1,005056ba-7cb6-1ed2-bceb-5332ab168d12,famila Tankstelle,FAMILA,Pascalstrasse,9,25442,Quickborn,53.742150,9.941240
2,005056ba-7cb6-1ed2-bceb-573c18314d16,star Tankstelle,STAR,Riehler Strasse,240,50735,Köln,50.961800,6.980070
3,005056ba-7cb6-1ed2-bceb-662ba1a94d1f,star Tankstelle,STAR,BAB 10 / Seeberg Ost,NaN,15345,Altlandsberg,52.550160,13.682120
4,005056ba-7cb6-1ed2-bceb-6f7b23564d23,star Tankstelle,STAR,Duisburger Straße,130,47166,Duisburg,51.489790,6.783730
